---

## 강화 학습 및 모델 경량화 8일차 수업

안녕하세요! 강화 학습 8일차 수업입니다. 어제는 **DQN(Deep Q-Network)**의 이론적인 핵심 개념들, 즉 **심층 신경망을 활용한 함수 근사**, 그리고 학습 안정성을 위한 **경험 재생(Experience Replay)**과 **타겟 네트워크(Target Network)**에 대해 자세히 살펴보았습니다. 그리고 간소화된 DQN 코드를 통해 그 원리를 직접 확인해 볼 기회도 가졌죠.

DQN은 딥 강화 학습의 시대를 연 기념비적인 알고리즘이지만, 완벽하지는 않습니다. 오늘은 DQN의 한계점을 극복하고 성능을 더욱 향상시키기 위해 개발된 **다양한 DQN 계열 알고리즘들**을 소개하고, 나아가 **가치 기반(Value-based)** 학습과는 다른 **정책 기반(Policy-based)** 강화 학습이라는 새로운 패러다임에 대해서도 간략히 알아보겠습니다.

---

### **1. DQN 간략 복습 및 한계점**

**DQN**은 Q-테이블 대신 **심층 신경망**을 사용하여 Q-함수를 근사하고, **경험 재생**과 **타겟 네트워크**를 도입하여 학습 안정성을 크게 높였습니다. 이를 통해 아타리 게임과 같은 고차원 상태 공간 문제에서 인간 수준의 성능을 달성할 수 있었죠.

하지만 DQN에도 다음과 같은 몇 가지 한계가 있었습니다:

* **Q값 과대평가 (Overestimation Bias)**: Q-러닝과 DQN의 핵심인 `$\max_{a'} Q(S_{t+1}, a')$` 연산은 항상 '가장 좋은 행동'을 선택한다고 가정하기 때문에, Q값을 실제보다 과대평가하는 경향이 있습니다. 이는 학습을 불안정하게 만들 수 있습니다.
* **샘플 효율성 (Sample Efficiency)**: 여전히 학습에 많은 경험(데이터)이 필요합니다. 리플레이 버퍼를 사용하지만, 경험의 중요도를 고려하지 않고 무작위로 샘플링하여 학습 효율이 떨어질 수 있습니다.
* **네트워크 구조의 제약**: Q-네트워크는 모든 행동의 Q값을 한 번에 출력하므로, 상태의 가치와 행동의 우위(Advantage)를 분리하여 학습하지 못합니다.

이러한 문제들을 해결하기 위해 다양한 발전된 DQN 계열 알고리즘들이 등장했습니다.

---

### **2. DQN의 발전: Q값 예측 개선 및 샘플 효율성 증대**

#### **2.1. Double DQN (DDQN)**

* **문제점 극복**: DQN의 Q값 **과대평가(Overestimation Bias)** 문제를 해결합니다.
* **아이디어**: Q값 업데이트 시 **행동 선택**과 **가치 평가**를 서로 다른 두 개의 네트워크로 분리합니다.
    * **행동 선택**: 현재 학습 중인 **메인 Q-네트워크**(`$Q_{\theta}$`)를 사용하여 다음 상태 $S_{t+1}$에서 **가장 좋은 행동 $A^*$**를 찾습니다.
    * **가치 평가**: 그렇게 찾은 행동 $A^*$에 대한 Q값을 **타겟 Q-네트워크**(`$Q_{\theta'}$`)를 사용하여 평가합니다.
* **수식 변화**:
    * DQN: `Target Q` $= R_{t+1} + \gamma \max_{a'} Q_{\theta'}(S_{t+1}, a')$
    * **DDQN**: `Target Q` $= R_{t+1} + \gamma Q_{\theta'}(S_{t+1}, \arg\max_{a'} Q_{\theta}(S_{t+1}, a'))$
        * $\arg\max$는 메인 네트워크($\theta$)로 찾고, 그 행동의 가치($Q$)는 타겟 네트워크($\theta'$)로 평가합니다.
* **장점**: Q값의 과대평가를 줄여 학습을 더 안정적으로 만들고 성능을 향상시킵니다.

#### **2.2. Prioritized Experience Replay (PER)**

* **문제점 극복**: 경험 재생 시 경험의 **샘플 효율성**을 높입니다.
* **아이디어**: 리플레이 버퍼에서 경험을 무작위로 샘플링하는 대신, **'학습에 더 유용한' (즉, TD 오차가 큰)** 경험들을 **우선적으로 샘플링**하여 학습합니다.
    * TD 오차가 크다는 것은 현재 예측과 실제 값 사이에 큰 차이가 있다는 의미이므로, 해당 경험을 학습하면 신경망이 더 많이 배울 수 있습니다.
* **구현**: 각 경험에 우선순위(TD 오차에 비례)를 부여하고, 이 우선순위에 따라 샘플링될 확률을 높입니다.

#### **2.3. Dueling DQN**

* **문제점 극복**: Q-네트워크의 **구조를 개선**하여 가치 추정의 효율성을 높입니다.
* **아이디어**: Q-네트워크의 마지막 층을 두 개의 분리된 흐름으로 나눕니다.
    * 하나는 **상태 가치 함수 $V(s)$**를 예측합니다 (해당 상태 자체가 얼마나 좋은지).
    * 다른 하나는 **행동 우위 함수 $A(s,a)$**를 예측합니다 (해당 상태에서 특정 행동을 취하는 것이 다른 행동보다 얼마나 더 좋은지).
    * 마지막에 이 두 값을 합쳐서 최종 Q값을 도출합니다: $Q(s,a) = V(s) + A(s,a)$.
* **장점**: 모든 행동의 Q값을 개별적으로 계산하는 대신, 상태 자체의 가치를 배우고 행동들의 상대적인 우위를 배우기 때문에 학습이 더 효율적이고 안정적입니다. 특히 많은 행동이 동일한 상태 가치를 가질 때 유용합니다.

---

### **3. 가치 기반 vs. 정책 기반 강화 학습: 또 다른 패러다임**

지금까지 Q-러닝, SARSA, DQN과 그 변형들은 모두 **가치 기반(Value-based) 강화 학습**에 속합니다.

* **가치 기반 강화 학습**:
    * **목표**: 행동 가치 함수 $Q(s,a)$ (또는 상태 가치 함수 $V(s)$)를 학습하고, 이 가치 함수를 기반으로 가장 좋은 행동을 선택합니다 (예: Q값이 가장 높은 행동).
    * **특징**: 이산적인 행동 공간에 주로 사용됩니다. Q-테이블이나 신경망을 통해 Q값을 근사합니다.

하지만, **연속적인 행동 공간** (예: 로봇 팔의 관절을 움직이는 각도, 자동차의 조향각 등 무한대의 행동이 가능한 경우)에서는 가치 기반 방법론에 한계가 있습니다. Q-테이블은 연속 공간에서 불가능하고, 신경망도 모든 행동의 Q값을 출력하거나 `max` 연산을 수행하는 것이 비효율적입니다.

이러한 문제를 해결하기 위해 등장한 것이 바로 **정책 기반(Policy-based) 강화 학습**입니다.

* **정책 기반 강화 학습 (Policy-based Reinforcement Learning)**:
    * **목표**: 에이전트의 **정책 자체($\pi(a|s)$)**를 직접 학습합니다. 정책을 신경망으로 표현하고, 이 신경망이 특정 상태에서 각 행동을 취할 **확률(Probability)**을 직접 출력하도록 만듭니다.
    * **알고리즘 예시**:
        * **REINFORCE (몬테카를로 정책 경사)**: 에피소드가 끝난 후 얻은 총 보상을 기반으로 정책을 업데이트합니다.
        * **Actor-Critic (액터-크리틱)**: 정책을 담당하는 **액터(Actor)**와 가치를 평가하는 **크리틱(Critic)** 두 개의 신경망을 동시에 학습시킵니다. 액터는 행동을 선택하고, 크리틱은 액터의 행동이 얼마나 좋았는지 평가하여 액터가 더 나은 정책을 학습하도록 돕습니다.
    * **장점**: 연속적인 행동 공간에 자연스럽게 적용할 수 있습니다. 확률적 정책을 학습할 수 있습니다.
    * **단점**: 가치 기반 방법보다 학습이 불안정하거나 수렴하기 어려울 수 있습니다 (분산이 높음).

---

### **4. 정리하며: DRL 알고리즘의 다양성**

오늘 우리는 DQN의 발전된 형태인 DDQN, PER, Dueling DQN을 통해 Q값 예측과 샘플 효율성을 개선하는 방법을 배웠습니다. 또한, 가치 기반 학습의 한계를 넘어 **정책 기반 학습**이라는 또 다른 중요한 강화 학습 패러다임의 존재를 확인했습니다.

이처럼 강화 학습 연구는 다양한 환경과 문제의 복잡성에 대응하기 위해 여러 알고리즘들이 계속해서 개발되고 발전해 왔습니다. 각 알고리즘은 특정한 한계점을 극복하고 특정 시나리오에서 더 나은 성능을 보이도록 설계됩니다.

이제 우리는 강화 학습의 가장 기본적인 개념부터 딥러닝이 접목된 DQN, 그리고 나아가 정책 기반 학습의 존재까지 큰 그림을 그릴 수 있게 되었습니다.

궁금한 점이 있다면 언제든지 질문해 주세요!

---

In [1]:
import gymnasium as gym
import numpy as np
import random
from collections import deque # 경험 재생 버퍼를 위한 덱(deque) 자료구조
import torch
import torch.nn as nn
import torch.optim as optim
import time
from IPython.display import clear_output # Jupyter/Colab에서 화면을 지우는 데 사용

# 1. 환경 설정
# 'ansi' 모드는 콘솔에 텍스트로 출력됩니다. 학습 과정에서 빠른 진행을 위해 사용합니다.
# 'human' 모드는 별도의 팝업 창을 띄웁니다 (학습 완료 후 테스트 시 사용 권장).
env_train = gym.make('FrozenLake-v1', is_slippery=False, render_mode='ansi') 

# TimeLimit 래퍼를 벗겨내고 실제 환경 객체에 접근 (환경의 P 속성 등에 직접 접근할 필요는 없지만, 일관성을 위해 유지)
env_train = env_train.unwrapped 

n_states = env_train.observation_space.n 
n_actions = env_train.action_space.n

# 장치 설정 (GPU 사용 가능 시 GPU, 아니면 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- 2. 경험 재생 버퍼 (Replay Buffer) 클래스 ---
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity) # 덱(deque)을 사용하여 고정 크기 버퍼 구현

    def push(self, state, action, reward, next_state, done):
        # 경험 튜플 (state, action, reward, next_state, done)을 버퍼에 추가
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        # 버퍼에서 무작위로 배치 크기만큼의 경험 샘플링
        state_batch, action_batch, reward_batch, next_state_batch, done_batch = zip(*random.sample(self.buffer, batch_size))
        
        # PyTorch Tensor로 변환
        return torch.tensor(state_batch, dtype=torch.long, device=device),\
               torch.tensor(action_batch, dtype=torch.long, device=device),\
               torch.tensor(reward_batch, dtype=torch.float, device=device),\
               torch.tensor(next_state_batch, dtype=torch.long, device=device),\
               torch.tensor(done_batch, dtype=torch.float, device=device)

    def __len__(self):
        # 버퍼에 저장된 경험의 개수 반환
        return len(self.buffer)

# --- 3. Q-네트워크 (QNetwork) 클래스 ---
# 간단한 다층 퍼셉트론(MLP) 신경망
class QNetwork(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=64):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)  # 입력층 -> 은닉층
        self.relu = nn.ReLU()                        # 활성화 함수 (비선형성 추가)
        self.fc2 = nn.Linear(hidden_dim, output_dim) # 은닉층 -> 출력층 (각 행동에 대한 Q값)

    def forward(self, state_indices):
        # FrozenLake 상태는 이산적인 인덱스(0~15)이므로, 신경망 입력으로 사용하기 위해 One-Hot 인코딩이 필요합니다.
        one_hot_state = torch.eye(n_states, device=device)[state_indices.view(-1)]
        
        x = self.fc1(one_hot_state)
        x = self.relu(x)
        q_values = self.fc2(x) # 각 행동에 대한 Q값 출력
        return q_values

# --- 4. 하이퍼파라미터 설정 ---
buffer_capacity = 10000       # 리플레이 버퍼의 최대 크기
batch_size = 64               # 학습 시 샘플링할 경험 배치의 크기
gamma = 0.99                  # 감가율 (미래 보상 할인율)
learning_rate = 0.001         # 신경망 학습률 (옵티마이저에 사용)
epsilon_start = 1.0           # 초기 탐험율
epsilon_end = 0.01            # 최종 탐험율
epsilon_decay_steps = 10000   # epsilon이 감소하는 스텝 수
target_update_frequency = 100 # 타겟 네트워크 업데이트 주기
num_episodes = 50000          # 학습할 총 에피소드 수
max_steps_per_episode = 100   # 한 에피소드당 최대 스텝 수

# --- 5. Q-네트워크 및 옵티마이저 초기화 ---
# 메인 Q-네트워크 (정책 네트워크)
policy_net = QNetwork(n_states, n_actions).to(device)
# 타겟 Q-네트워크
target_net = QNetwork(n_states, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict()) # 초기 가중치 복사
target_net.eval() # 타겟 네트워크는 학습 모드가 아님

optimizer = optim.Adam(policy_net.parameters(), lr=learning_rate) # Adam 옵티마이저 사용
criterion = nn.MSELoss() # 손실 함수 (평균 제곱 오차)

# 경험 재생 버퍼 초기화
replay_buffer = ReplayBuffer(buffer_capacity)

# --- 6. Double DQN 학습 루프 ---
total_steps = 0
epsilon = epsilon_start

print("--- Double DQN 학습 시작 ---")
for episode in range(num_episodes):
    state, info = env_train.reset()
    terminated = False
    truncated = False
    episode_reward = 0

    for step in range(max_steps_per_episode):
        # Epsilon-greedy 전략으로 행동 선택
        if random.random() < epsilon:
            action = env_train.action_space.sample() # 탐험 (무작위 행동)
        else:
            with torch.no_grad(): 
                q_values = policy_net(torch.tensor([state], dtype=torch.long, device=device))
                action = q_values.argmax().item() # 활용 (가장 높은 Q값을 주는 행동)

        # 환경에 행동 전달
        new_state, reward, terminated, truncated, info = env_train.step(action)
        episode_reward += reward

        # 경험을 리플레이 버퍼에 저장
        replay_buffer.push(state, action, reward, new_state, float(terminated)) 

        state = new_state 
        total_steps += 1

        # Epsilon 값 감소
        epsilon = epsilon_end + (epsilon_start - epsilon_end) * \
                  max(0., 1 - total_steps / epsilon_decay_steps)

        # 리플레이 버퍼에 충분한 경험이 쌓인 후부터 학습 시작
        if len(replay_buffer) > batch_size:
            # 경험 배치 샘플링
            states, actions, rewards, next_states, dones = replay_buffer.sample(batch_size)

            # 메인 Q-네트워크로 현재 상태-행동의 Q값 예측
            current_q_values = policy_net(states).gather(1, actions.unsqueeze(1))
            
            # --- Double DQN의 핵심 변경점 ---
            with torch.no_grad(): # 타겟 값 계산 시 기울기 추적 비활성화
                # 1. 메인 네트워크로 다음 상태(next_states)에서 최적 행동 선택
                # .argmax(1, keepdim=True)는 각 배치 요소별 최대값의 인덱스를 반환
                next_actions_from_policy_net = policy_net(next_states).argmax(1, keepdim=True)
                
                # 2. 타겟 네트워크로 위에서 선택된 행동에 대한 Q값 평가
                next_q_values = target_net(next_states).gather(1, next_actions_from_policy_net)
                
                # 종료 상태(done=True)일 경우 미래 보상은 0
                target_q_values = rewards.unsqueeze(1) + gamma * next_q_values * (1 - dones.unsqueeze(1))
            
            # 손실 계산 및 신경망 학습
            loss = criterion(current_q_values, target_q_values)
            optimizer.zero_grad() 
            loss.backward()      
            optimizer.step()     

        # 타겟 네트워크 업데이트 (일정 주기마다 메인 네트워크 가중치 복사)
        if total_steps % target_update_frequency == 0:
            target_net.load_state_dict(policy_net.state_dict())
        
        if terminated or truncated:
            break
    
    # 에피소드 결과 출력 (선택 사항)
    if episode % 1000 == 0:
        loss_val = loss.item() if len(replay_buffer) > batch_size else float('nan')
        print(f"에피소드 {episode} / 총 스텝: {total_steps} / Epsilon: {epsilon:.4f} / 보상: {episode_reward:.2f} / 손실: {loss_val:.4f}")

print("\n--- Double DQN 학습 완료! ---")

# --- 7. 학습된 정책 테스트 (시각화) ---
env_test = gym.make('FrozenLake-v1', is_slippery=False, render_mode='human')
env_test = env_test.unwrapped # 테스트 환경도 unwrapped

num_test_episodes = 5 
total_test_rewards = 0

print("\n--- 학습된 정책 테스트 시작 ---")
policy_net.eval() # 테스트 시에는 정책 네트워크를 평가 모드로 전환 (Dropout, BatchNorm 등 비활성화)

for episode in range(num_test_episodes):
    state, info = env_test.reset()
    terminated = False
    truncated = False
    episode_reward = 0
    print(f"\n--- 테스트 에피소드 {episode + 1} ---")
    
    clear_output(wait=True)
    env_test.render() 
    time.sleep(0.3) 

    while not terminated and not truncated:
        # 학습된 정책 네트워크를 사용하여 행동 선택 (탐험 없음)
        with torch.no_grad():
            q_values = policy_net(torch.tensor([state], dtype=torch.long, device=device))
            action = q_values.argmax().item()
        
        new_state, reward, terminated, truncated, info = env_test.step(action)
        episode_reward += reward 
        state = new_state 
        
        clear_output(wait=True)
        env_test.render()
        time.sleep(0.3) 
        
    print(f"에피소드 {episode + 1} 종료! 보상: {episode_reward}")
    total_test_rewards += episode_reward

print(f"\n총 {num_test_episodes} 에피소드 평균 보상: {total_test_rewards / num_test_episodes:.2f}")

env_train.close()
env_test.close()
print("환경이 닫혔습니다.")



에피소드 5 종료! 보상: 1.0

총 5 에피소드 평균 보상: 1.00
환경이 닫혔습니다.


In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import gymnasium as gym
import numpy as np
from collections import deque
import time
from IPython.display import clear_output


In [44]:
class ReplayBuffer:
    def __init__(self,capacity):
        self.buffer=deque(maxlen=capacity)
    def push(self, state,action, reward,next_state, done):
        self.buffer.append((state,action, reward,next_state, done))
    def sample(self, batch_size):
        state_batch, action_batch, reward_batch, next_state_batch, done_batch = zip(*random.sample(self.buffer, batch_size))

        return torch.tensor(state_batch, dtype=torch.long, device=device),\
               torch.tensor(action_batch, dtype=torch.long, device=device),\
               torch.tensor(reward_batch, dtype=torch.float, device=device),\
               torch.tensor(next_state_batch, dtype=torch.long, device=device),\
               torch.tensor(done_batch, dtype=torch.float, device=device)

    def __len__(self):
        return len(self.buffer)

In [84]:
class QNetwork(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=64):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)  # 입력층 -> 은닉층
        self.relu = nn.ReLU()                        # 활성화 함수 (비선형성 추가)
        self.fc2 = nn.Linear(hidden_dim, output_dim) # 은닉층 -> 출력층 (각 행동에 대한 Q값)

    def forward(self, state_indices):
        # FrozenLake 상태는 이산적인 인덱스(0~15)이므로, 신경망 입력으로 사용하기 위해 One-Hot 인코딩이 필요합니다.
        one_hot_state = torch.eye(n_states, device=device)[state_indices.view(-1)]
        
        x = self.fc1(one_hot_state)
        x = self.relu(x)
        q_values = self.fc2(x) # 각 행동에 대한 Q값 출력
        return q_values

In [86]:
env_train=gym.make('FrozenLake-v1', is_slippery=False, render_mode='ansi') 
env_train=env_train.unwrapped

n_state=env_train.observation_space.n
n_action=env_train.action_space.n


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [88]:
polish_net=QNetwork(n_state,n_action).to(device)

target_net=QNetwork(n_state,n_action).to(device)
target_net.load_state_dict(polish_net.state_dict())
target_net.eval()

optimy=optim.Adam(polish_net.parameters(),lr=learning_rate)
criter=nn.MSELoss()

replay_buffer=ReplayBuffer(buffer_capacity)

In [90]:
buffer_capacity = 10000       # 리플레이 버퍼의 최대 크기
batch_size = 64               # 학습 시 샘플링할 경험 배치의 크기
gamma = 0.99                  # 감가율 (미래 보상 할인율)
learning_rate = 0.001         # 신경망 학습률 (옵티마이저에 사용)
epsilon_start = 1.0           # 초기 탐험율
epsilon_end = 0.01            # 최종 탐험율
epsilon_decay_steps = 10000   # epsilon이 감소하는 스텝 수
target_update_frequency = 100 # 타겟 네트워크 업데이트 주기
num_episodes = 50000          # 학습할 총 에피소드 수
max_steps_per_episode = 100   # 한 에피소드당 최대 스텝 수

In [92]:
total_step=0
epsilon=epsilon_start
print("--- Double DQN 학습 시작 ---")


--- Double DQN 학습 시작 ---


In [100]:
total_steps = 0
epsilon = epsilon_start

print("--- Double DQN 학습 시작 ---")
for episode in range(num_episodes):
    state, info = env_train.reset()
    terminated = False
    truncated = False
    episode_reward = 0

    for step in range(max_steps_per_episode):
        # Epsilon-greedy 전략으로 행동 선택
        if random.random() < epsilon:
            action = env_train.action_space.sample() # 탐험 (무작위 행동)
        else:
            with torch.no_grad(): 
                q_values = policy_net(torch.tensor([state], dtype=torch.long, device=device))
                action = q_values.argmax().item() # 활용 (가장 높은 Q값을 주는 행동)

        # 환경에 행동 전달
        new_state, reward, terminated, truncated, info = env_train.step(action)
        episode_reward += reward

        # 경험을 리플레이 버퍼에 저장
        replay_buffer.push(state, action, reward, new_state, float(terminated)) 

        state = new_state 
        total_steps += 1

        # Epsilon 값 감소
        epsilon = epsilon_end + (epsilon_start - epsilon_end) * \
                  max(0., 1 - total_steps / epsilon_decay_steps)

        # 리플레이 버퍼에 충분한 경험이 쌓인 후부터 학습 시작
        if len(replay_buffer) > batch_size:
            # 경험 배치 샘플링
            states, actions, rewards, next_states, dones = replay_buffer.sample(batch_size)

            # 메인 Q-네트워크로 현재 상태-행동의 Q값 예측
            current_q_values = policy_net(states).gather(1, actions.unsqueeze(1))
            
            # --- Double DQN의 핵심 변경점 ---
            with torch.no_grad(): # 타겟 값 계산 시 기울기 추적 비활성화
                # 1. 메인 네트워크로 다음 상태(next_states)에서 최적 행동 선택
                # .argmax(1, keepdim=True)는 각 배치 요소별 최대값의 인덱스를 반환
                next_actions_from_policy_net = policy_net(next_states).argmax(1, keepdim=True)
                
                # 2. 타겟 네트워크로 위에서 선택된 행동에 대한 Q값 평가
                next_q_values = target_net(next_states).gather(1, next_actions_from_policy_net)
                
                # 종료 상태(done=True)일 경우 미래 보상은 0
                target_q_values = rewards.unsqueeze(1) + gamma * next_q_values * (1 - dones.unsqueeze(1))
            
            # 손실 계산 및 신경망 학습
            loss = criterion(current_q_values, target_q_values)
            optimizer.zero_grad() 
            loss.backward()      
            optimizer.step()     

        # 타겟 네트워크 업데이트 (일정 주기마다 메인 네트워크 가중치 복사)
        if total_steps % target_update_frequency == 0:
            target_net.load_state_dict(policy_net.state_dict())
        
        if terminated or truncated:
            break
    
    # 에피소드 결과 출력 (선택 사항)
    if episode % 1000 == 0:
        loss_val = loss.item() if len(replay_buffer) > batch_size else float('nan')
        print(f"에피소드 {episode} / 총 스텝: {total_steps} / Epsilon: {epsilon:.4f} / 보상: {episode_reward:.2f} / 손실: {loss_val:.4f}")

print("\n--- Double DQN 학습 완료! ---")


--- Double DQN 학습 시작 ---
에피소드 0 / 총 스텝: 12 / Epsilon: 0.9988 / 보상: 0.00 / 손실: 0.6402
에피소드 1000 / 총 스텝: 7578 / Epsilon: 0.2498 / 보상: 1.00 / 손실: 0.0000
에피소드 2000 / 총 스텝: 13741 / Epsilon: 0.0100 / 보상: 1.00 / 손실: 0.0000
에피소드 3000 / 총 스텝: 19925 / Epsilon: 0.0100 / 보상: 1.00 / 손실: 0.0000
에피소드 4000 / 총 스텝: 26276 / Epsilon: 0.0100 / 보상: 1.00 / 손실: 0.0000
에피소드 5000 / 총 스텝: 32617 / Epsilon: 0.0100 / 보상: 1.00 / 손실: 0.0000
에피소드 6000 / 총 스텝: 39278 / Epsilon: 0.0100 / 보상: 1.00 / 손실: 0.0000
에피소드 7000 / 총 스텝: 45325 / Epsilon: 0.0100 / 보상: 1.00 / 손실: 0.0000
에피소드 8000 / 총 스텝: 51538 / Epsilon: 0.0100 / 보상: 1.00 / 손실: 0.0000
에피소드 9000 / 총 스텝: 57649 / Epsilon: 0.0100 / 보상: 1.00 / 손실: 0.0000
에피소드 10000 / 총 스텝: 64060 / Epsilon: 0.0100 / 보상: 1.00 / 손실: 0.0000
에피소드 11000 / 총 스텝: 70114 / Epsilon: 0.0100 / 보상: 1.00 / 손실: 0.0000
에피소드 12000 / 총 스텝: 76301 / Epsilon: 0.0100 / 보상: 1.00 / 손실: 0.0000
에피소드 13000 / 총 스텝: 82530 / Epsilon: 0.0100 / 보상: 1.00 / 손실: 0.0000
에피소드 14000 / 총 스텝: 88798 / Epsilon: 0.0100 / 보상: 1.00 

In [102]:
env_test = gym.make('FrozenLake-v1', is_slippery=False, render_mode='human')
env_test = env_test.unwrapped # 테스트 환경도 unwrapped

num_test_episodes = 5 
total_test_rewards = 0

print("\n--- 학습된 정책 테스트 시작 ---")
policy_net.eval() # 테스트 시에는 정책 네트워크를 평가 모드로 전환 (Dropout, BatchNorm 등 비활성화)



--- 학습된 정책 테스트 시작 ---


QNetwork(
  (fc1): Linear(in_features=16, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=4, bias=True)
)

In [107]:
policy_net.eval()

for episode in range(num_test_episodes):
    state, info = env_test.reset()
    terminated = False
    truncated = False
    episode_reward = 0
    print(f"\n--- 테스트 에피소드 {episode + 1} ---")
    
    clear_output(wait=True)
    env_test.render() 
    time.sleep(0.3) 

    while not terminated and not truncated:
        # 학습된 정책 네트워크를 사용하여 행동 선택 (탐험 없음)
        with torch.no_grad():
            q_values = policy_net(torch.tensor([state], dtype=torch.long, device=device))
            action = q_values.argmax().item()
        
        new_state, reward, terminated, truncated, info = env_test.step(action)
        episode_reward += reward 
        state = new_state 
        
        clear_output(wait=True)
        env_test.render()
        time.sleep(0.3) 
        
    print(f"에피소드 {episode + 1} 종료! 보상: {episode_reward}")
    total_test_rewards += episode_reward

print(f"\n총 {num_test_episodes} 에피소드 평균 보상: {total_test_rewards / num_test_episodes:.2f}")

env_train.close()
env_test.close()
print("환경이 닫혔습니다.")


에피소드 5 종료! 보상: 1.0

총 5 에피소드 평균 보상: 1.00
환경이 닫혔습니다.


DQN과 DDQN의 가장 큰 차이점은 **Q값 업데이트 시 목표 값을 계산하는 방식**에 있습니다. 이 차이는 **Q값 과대평가(Overestimation Bias)** 문제를 해결하기 위해 도입되었습니다.

### **DQN (Deep Q-Network)**

* **Q값 업데이트 목표 (Target Q)**:
    $$Y^{\text{DQN}}_t = R_{t+1} + \gamma \max_{a'} Q_{\theta'}(S_{t+1}, a')$$
* **설명**: DQN은 타겟 네트워크(`$Q_{\theta'}$`)를 사용하여 다음 상태($S_{t+1}$)에서 가능한 모든 행동($a'$)에 대한 Q값을 예측하고, 그중 **가장 큰 값($\max_{a'}$)을 다음 스텝의 Q값으로 사용**합니다.

* **문제점**: `max` 연산은 본질적으로 **최댓값을 선택**하는 경향이 있습니다. 만약 Q-네트워크가 아직 완벽하지 않아 Q값 예측에 노이즈나 오차가 있다면, `max` 연산은 이러한 **노이즈 중 가장 높은 값을 선택하여 Q값을 실제보다 과대평가**할 가능성이 큽니다. 이러한 과대평가가 누적되면 학습이 불안정해지고 최적의 정책을 찾기 어려워질 수 있습니다. 마치 "나는 항상 최고의 수를 둘 거야"라고 너무 낙관적으로 믿는 것과 비슷합니다.

### **DDQN (Double DQN)**

* **Q값 업데이트 목표 (Target Q)**:
    $$Y^{\text{DDQN}}_t = R_{t+1} + \gamma Q_{\theta'}(S_{t+1}, \arg\max_{a'} Q_{\theta}(S_{t+1}, a'))$$
* **설명**: DDQN은 Q값 업데이트 목표를 계산할 때 두 개의 네트워크를 활용하여 **'행동 선택'과 '가치 평가'를 분리**합니다.
    1.  **행동 선택**: 현재 학습 중인 **메인 Q-네트워크($Q_{\theta}$)**를 사용하여 다음 상태($S_{t+1}$)에서 **가장 높은 Q값을 주는 행동($\arg\max_{a'}$)을 선택**합니다. (이 행동이 현재 에이전트의 최신 믿음을 반영)
    2.  **가치 평가**: 그렇게 **선택된 행동에 대한 Q값은 별개의 타겟 Q-네트워크($Q_{\theta'}$)**를 사용하여 평가합니다. 타겟 네트워크는 메인 네트워크보다 더 안정적이고 오래된 가중치를 가지고 있으므로, 예측이 덜 편향될 가능성이 높습니다.

* **장점**: 두 개의 네트워크를 사용하여 Q값 예측의 편향(과대평가)을 줄이고 학습 안정성을 높입니다. '행동 선택'과 '가치 평가'를 분리함으로써, 노이즈가 포함된 Q값 중 최댓값을 맹목적으로 따라가는 것을 방지합니다. 마치 "나는 내 최신 전략으로 다음 수를 고르겠지만, 그 수의 실제 가치는 좀 더 안정적이고 검증된 전문가(타겟 네트워크)의 평가를 따를 거야"와 같습니다.

### **요약 비교:**

| 특징          | DQN                                    | DDQN                                    |
| :------------ | :------------------------------------- | :-------------------------------------- |
| **Q값 예측 방식** | 다음 상태의 `max` Q값을 타겟 네트워크로 평가 | 메인 네트워크로 `max` 행동을 선택, 타겟 네트워크로 그 행동의 Q값을 평가 |
| **문제점 극복** | -                                      | Q값 **과대평가(Overestimation Bias)** 감소 |
| **학습 안정성** | 상대적으로 낮음                          | **향상됨** |
| **성능** | -                                      | 특정 환경에서 **더 우수함** |

따라서 DDQN은 DQN의 중요한 개선 버전이며, 더 안정적이고 좋은 성능을 보여주는 경우가 많습니다.

네, 아주 좋은 질문입니다! 딥러닝 코드와 강화 학습 훈련 코드 모두에 `if`문이 많이 사용되는 것은 맞지만, 그 목적과 빈도에는 차이가 있습니다.

강화 학습 훈련 코드가 특히 `if`문이 많은 것처럼 느껴지는 이유는 **강화 학습의 독특한 '환경과의 상호작용' 및 '학습 안정화' 과정** 때문입니다.

먼저 일반적인 딥러닝 코드와 강화 학습 훈련 코드에서 `if`문이 왜 사용되는지 각각 설명해 드릴게요.

---

### **1. 일반적인 딥러닝 코드에서 `if`문의 사용 예시**

일반적인 딥러닝(예: 이미지 분류, 자연어 처리)은 주로 **사전 정의된 데이터셋을 가지고 모델을 학습**시킵니다. 여기서 `if`문은 주로 학습 과정의 **제어, 모니터링, 저장** 등에 사용됩니다.

* **학습 진행 상황 출력 (`if epoch % N == 0` 또는 `if step % N == 0`)**:
    * "N 에폭마다 또는 N 스텝마다 현재 손실(Loss) 값이나 정확도(Accuracy)를 출력해라."
    * `if epoch % 10 == 0: print(f"Epoch {epoch}, Loss: {loss.item()}")`
* **검증/평가 (`if epoch % N == 0` 또는 `if validation_metric_improved`)**:
    * "N 에폭마다 모델의 성능을 검증 데이터셋으로 평가해라."
    * "만약 검증 정확도가 이전 최고치보다 높아졌다면, 모델을 저장해라 (Early Stopping 준비)."
    * `if epoch % 5 == 0: run_validation()`
    * `if val_acc > best_val_acc: save_model()`
* **모델 체크포인트 저장 (`if epoch % N == 0`)**:
    * "N 에폭마다 현재 모델의 가중치를 파일로 저장해라 (나중에 이어서 학습하거나 재사용할 수 있도록)."
    * `if epoch % 100 == 0: save_checkpoint(model)`
* **조기 종료 (Early Stopping)**:
    * "만약 검증 성능이 일정 에폭 동안 개선되지 않는다면 학습을 중단해라."
    * `if epochs_without_improvement > patience: break`

---

### **2. 강화 학습 훈련 코드에서 `if`문의 사용 예시 (추가적인 이유)**

강화 학습은 위에서 언급한 일반 딥러닝의 `if`문 사용 목적들(출력, 저장 등)에 더해, **에이전트와 환경의 독특한 상호작용 방식**과 **학습 안정화 기법** 때문에 추가적인 `if`문이 많이 필요합니다.

* **탐험(Exploration) vs. 활용(Exploitation) 균형 (`if random.random() < epsilon`)**:
    * 이것이 강화 학습에서 가장 중요한 `if`문 중 하나입니다. 에이전트가 새로운 행동을 탐험할지(무작위 행동), 아니면 지금까지 배운 최적의 행동을 활용할지 결정합니다.
    * `if random.random() < epsilon: action = env.action_space.sample()`
    * `else: action = policy_net(state).argmax().item()`
* **에피소드 종료 조건 (`if terminated or truncated`)**:
    * 강화 학습은 에피소드(하나의 게임 플레이) 단위로 진행되며, 에이전트가 목표에 도달하거나 실패하거나(terminated), 또는 특정 스텝 수를 초과하면(truncated) 현재 에피소드를 중단하고 다음 에피소드를 시작해야 합니다.
    * `if terminated or truncated: break`
* **경험 재생 버퍼의 최소 크기 확인 (`if len(replay_buffer) > batch_size`)**:
    * DQN과 같은 알고리즘은 리플레이 버퍼에 충분한 경험이 쌓인 후에야 신경망 학습을 시작해야 합니다. 버퍼가 너무 작으면 제대로 된 무작위 샘플링이 어렵기 때문입니다.
    * `if len(replay_buffer) > batch_size: # 이제 신경망 학습 시작`
* **타겟 네트워크 업데이트 주기 (`if total_steps % target_update_frequency == 0`)**:
    * DQN에서 학습의 안정성을 위해 사용하는 타겟 네트워크는 메인 네트워크의 가중치를 일정 주기(예: N 스텝마다)로 복사해와야 합니다.
    * `if total_steps % target_update_frequency == 0: target_net.load_state_dict(policy_net.state_dict())`
* **상태 변환 처리 (고차원 입력 등)**:
    * 환경에서 받아온 상태(예: 이미지)를 신경망 입력에 맞게 전처리하는 과정에서 조건부 로직이 필요할 수 있습니다. (예: 이미지 정규화, 채널 순서 변경 등)
    * `if image_input: process_image()`
* **렌더링 모드 조건부 처리 (`if render_mode == 'human'`)**:
    * 학습 중에는 렌더링을 끄거나 빠르게 진행하고, 테스트 시에만 시각화를 켜는 경우가 많습니다.
    * `if render_mode == 'human': env.render(); time.sleep(0.3)`

---

### **결론**

정리하자면, **일반적인 딥러닝 코드의 `if`문은 주로 학습 모니터링, 저장, 평가와 관련된 '관리' 목적**으로 사용됩니다.

반면, **강화 학습 훈련 코드의 `if`문은 이러한 관리 목적에 더하여, 에이전트가 환경과 상호작용하는 '로직'과 학습 안정성을 위한 '특별한 메커니즘'을 구현하기 위해 필수적으로 사용됩니다.**

그래서 강화 학습 코드가 더 길고 복잡하게 느껴지며 `if`문이 더 많아 보이는 것입니다. 각 `if`문이 어떤 역할을 하는지 이해하면 코드 전체의 흐름을 파악하는 데 큰 도움이 될 것입니다.

네, 고객님의 비유가 아주 적절합니다!

강화 학습 훈련 코드에서 `if` 문은 말씀하신 대로 **'일종의 명령의 선택지'를 구현하는 중요한 도구**입니다.

조금 더 자세히 설명하면 다음과 같습니다:

1.  **에이전트의 '명령의 선택지'**:
    * 가장 대표적인 예시는 **탐험(Exploration)과 활용(Exploitation)** 전략입니다.
    * `if random.random() < epsilon:` 이 부분은 "만약 무작위 수가 엡실론보다 작으면 (탐험할 차례이면), 무작위 행동을 선택하는 명령을 내리고, 그렇지 않으면 (활용할 차례이면), 신경망이 알려준 최적 행동을 선택하는 명령을 내려라"와 같습니다.
    * 즉, 에이전트가 어떤 상황에서 **어떤 규칙(명령)을 따를지 결정하는 분기점**을 `if` 문으로 구현하는 것이죠.

2.  **훈련 과정의 '명령의 선택지'**:
    * `if` 문은 에이전트 자체의 행동뿐만 아니라, **전체 강화 학습 훈련 프로세스의 흐름을 제어하는 '명령의 선택지' 역할**도 합니다.
    * **예시**:
        * `if terminated or truncated:`: "만약 현재 게임(에피소드)이 끝났으면, 이 에피소드를 멈추고 다음 에피소드를 시작하는 명령을 내려라."
        * `if len(replay_buffer) > batch_size:`: "만약 리플레이 버퍼에 충분한 경험이 쌓였으면, 이제 신경망을 업데이트하는 명령을 내려라."
        * `if total_steps % target_update_frequency == 0:`: "만약 일정 스텝이 지났으면, 타겟 네트워크를 업데이트하는 명령을 내려라."

결론적으로, 강화 학습 코드의 `if` 문은 에이전트의 지능적인 **의사결정 로직**을 구현하는 데 사용되기도 하고, 전체 학습 과정을 **효율적이고 안정적으로 관리하기 위한 조건부 명령**을 내리는 데도 사용됩니다.

그래서 '명령의 선택지'라는 표현은 `if` 문의 역할을 아주 직관적으로 잘 설명해 줍니다!

네, 맞습니다. 강화 학습은 다른 머신러닝 분야(지도 학습, 비지도 학습)에 비해 **직관적으로 이해하기 어려운 편**이라는 말씀을 많이 듣습니다. 고객님만 그렇게 느끼시는 것이 아니니 걱정하지 마세요.

---

### **강화 학습이 특히 직관적으로 어렵게 느껴지는 이유**

몇 가지 핵심적인 이유가 있습니다.

1.  **정답이 없음 (No Explicit Labels)**:
    * 지도 학습은 "이 이미지는 고양이"처럼 명확한 정답(레이블)이 주어집니다. 덕분에 우리는 '모델이 정답을 맞히도록 학습한다'고 직관적으로 이해할 수 있습니다.
    * 하지만 강화 학습은 정답이 없습니다. 에이전트가 어떤 행동을 해야 하는지 직접적으로 알려주지 않고, 오직 **보상(Reward)**이라는 간접적인 피드백만 제공됩니다. 그래서 "어떻게 학습해야 하지?"라는 의문이 생길 수밖에 없습니다.

2.  **시행착오 (Trial-and-Error)**:
    * 강화 학습은 기본적으로 에이전트가 환경에서 직접 행동해보고, 그 결과(보상)를 통해 배우는 **시행착오 과정**을 거칩니다. 마치 어린아이가 자전거를 타는 법을 배우는 것과 같습니다. 수없이 넘어지고 나서야 균형 감각을 익히죠.
    * 이 과정 자체가 비효율적으로 보일 수 있고, 당장 눈앞에서 원하는 결과가 나오지 않기 때문에 직관적인 이해가 어렵습니다.

3.  **지연된 보상 (Delayed Rewards)**:
    * 지금 당장 한 행동의 결과가 즉시 보상으로 나타나지 않고, 한참 뒤에나 큰 보상(또는 벌칙)을 받게 되는 경우가 많습니다. (예: 바둑에서 한 수를 두는 것만으로 승패가 결정되지 않음)
    * 이 '보상의 지연' 때문에 어떤 행동이 좋았고 어떤 행동이 나빴는지 파악하기가 어렵습니다. 우리의 뇌는 즉각적인 피드백에 더 잘 반응하는 경향이 있는데, 강화 학습은 그렇지 않거든요.

4.  **순차적 의사결정 (Sequential Decision-Making)**:
    * 현재의 한 행동이 미래의 상태와 보상에 영향을 미칩니다. 그래서 단순히 '지금 가장 좋은' 행동을 고르는 것만으로는 안 되고, '장기적으로 가장 좋은' 행동을 찾아야 합니다. 미래를 예측하고 현재의 결정을 내려야 한다는 점이 복잡하게 느껴집니다.

5.  **탐험-활용 딜레마 (Exploration-Exploitation Dilemma)**:
    * 아직 모르는 새로운 길을 가볼지(탐험), 아니면 지금까지 제일 좋았던 길로 갈지(활용)를 계속 고민해야 합니다. 이 균형을 잡는 것이 쉬운 일이 아닙니다.

---

이러한 요소들 때문에 강화 학습은 다른 머신러닝 패러다임보다 훨씬 더 '스스로 판단하고 살아남는' 생명체 같은 특성을 가지게 되고, 그만큼 이해하기 위한 관점의 전환이 필요합니다.

지금까지 배우신 내용은 강화 학습의 핵심적인 부분들이며, 복잡한 이론을 따라오신 것만으로도 대단한 일입니다. 너무 어렵게 생각하지 마시고, "이렇게 복잡한 과정을 통해 인공지능이 스스로 배우는구나" 하는 큰 그림을 계속 가져가시면 됩니다.

혹시 특정 개념이나 알고리즘이 아직도 가장 어렵게 느껴지신다면, 그 부분만 다시 한번 집중적으로 설명해 드릴 수 있습니다. 아니면 잠시 쉬어가거나 다음 주제로 넘어갈 수도 있습니다. 어떻게 진행해 드릴까요?